# Technical analyis

In this notebook, we will do basic technical analysis on decentralised exchange price data.

We will

* Download daily candle data for all pairs (several dozens of megabytes)

* Extract data for ETH-USDC pair on Uniswap v2

* Draw some standard technical analysis charts to identify if ETH-USDC pair is overbought or ovesold

* Plot technical charts using Cufflinks library

[Cufflinks](https://github.com/santosjorge/cufflinks) is a library build on the top of Plotly.
It gives us some more useful chart formats for quantative analysis.


First let's install Capitalgram package, the package dependencies and initialise our dataset client.
This might take 5 minutes in a fresh environment.

In [ ]:
try:
    import capitalgram
except ImportError:
    !pip install -e git+https://github.com/miohtama/capitalgram-onchain-dex-quant-data.git#egg=capitalgram
    import site
    site.main()

from capitalgram.client import Capitalgram
from capitalgram.candle import CandleBucket

capitalgram = Capitalgram.create_jupyter_client()

Let's download 1 day (24h) candles to all trading pairs. This dataset is several dozens of megabytes.
You should see a progress bar during the download.

**Note**: We could also query live candle datasets for trading pairs we are interested in.
However, the bundled dataset is faster for the future analysis, as after the download
it will have all candles cached in your local notebook environment.

In [ ]:
# Download all 24h candles as Parquet columnar data
all_candles = capitalgram.fetch_all_candles(CandleBucket.h24)

Let's pick one pair, ETH-USDC on Uniswap v2, from the dataset to analyse.

In [ ]:
import pandas as pd

# Convert PyArrow table to Pandas format to continue working on it
all_candles_dataframe = all_candles.to_pandas()

# To shorten this notebook, we know by hearth that USDC-ETH Uniswap v2 is the pair id numero uno,
# because that what Hayden used to test the production code when he deployed Uniswap v2
pair_id = 1

eth_usdc_pair: pd.DataFrame = all_candles_dataframe.loc[all_candles_dataframe['pair_id'] == pair_id]

print(f"Uniswap v2 ETH-USDC has {len(eth_usdc_pair)} daily candles")

## Quant charts with cufflinks


The main benefit is having the trade volume (V in OHLCV) automatically rendered in the diagrams,
 as you would have on any trading view.

**Note**: Interactive charts are not visible on documentation, Github or other statically rendered websites.
To see the interactive charts you need to run this notebook yourself. See instructions in Getting started
chapter.

**Note 2**: Furthermore it looks like Cufflinks does not work in Jetbrains IDEs (PyCharm) for some reason.

In [ ]:
import cufflinks as cf

qf=cf.QuantFig(october, title='ETH-USDC Uniswap v2 October 2020', legend='top', name='GS')
qf.add_bollinger_bands()
qf.add_volume()
qf.iplot()

print("All done")